## Load libraries

In [180]:
# Libraries to work cross-platform
import os

# Libraries to work with dataset
# import numpy as np
import pandas as pd

# Libraries to pre-process data
import nltk
# from nltk.corpus import stopwords
import re
import string
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet
from rake_nltk import Rake
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
# import spacy_ke

# Libraries for monitoring operation process
from tqdm import tqdm

## Configurate and declare global variables

In [181]:
os_name = os.name

if os_name == 'nt':
    BASE_DIR = "E:/THIENDHB_GOOGLEDRIVE/MASTER TILBURG/THESIS/"
    INPUT_DIR = BASE_DIR + "DATASET/INPUT/"
    OUTPUT_DIR = BASE_DIR + "DATASET/OUTPUT/"
elif os_name == 'posix':
    BASE_DIR = "/media/pinkalinux/WORK/THIENDHB_GOOGLEDRIVE/MASTER TILBURG/THESIS/"
    INPUT_DIR = BASE_DIR + "DATASET/INPUT/"
    OUTPUT_DIR = BASE_DIR + "DATASET/OUTPUT/"

SEED = 6886

%matplotlib inline

## Import data

In [229]:
df_raw = pd.read_csv(INPUT_DIR + "refined_jobpost_data.csv")
pd.set_option('display.max_colwidth', None)

## Explore data

In [230]:
skill_columns = ["job_description", "job_requirement", "job_qualification"]
skill_df = df_raw[["job_id"] + skill_columns]
skill_df.head(2)

,job_id,job_description,job_requirement,job_qualification
0,1,"AMERIA Investment Consulting Company is seeking a\r\nChief Financial Officer. This position manages the company's fiscal and\r\nadministrative functions, provides highly responsible and technically\r\ncomplex staff assistance to the Executive Director. The work performed\r\nrequires a high level of technical proficiency in financial management\r\nand investment management, as well as management, supervisory, and\r\nadministrative skills.","- Supervises financial management and administrative staff, including\r\nassigning responsibilities, reviewing employees' work processes and\r\nproducts, counseling employees, giving performance evaluations, and\r\nrecommending disciplinary action;\r\n- Serves as member of management team participating in both strategic\r\nand operational planning for the company;\r\n- Directs and oversees the company's financial management activities,\r\nincluding establishing and monitoring internal controls, managing cash\r\nand investments, and managing the investment portfolio in collaboration\r\nwith the Investment team leader. This includes, but is not limited to,\r\nevaluation of investment risk, concentration risk, fund deployment\r\nlevels, adequacy of loss and liquidity reserves Assists investment team\r\nin development of proper documentation and internal systems;\r\n- Directs and oversees the annual budgeting process, including\r\ndeveloping projections for financial planning, and preparing budgets;\r\n- Prepares external and internal financial management reports, such as\r\naudited financial statements, tax returns, and reports for the board of\r\ndirectors and company staff;\r\n- Develops, implements, and maintains efficient and effective accounting\r\nsystems and controls to ensure compliance with national and\r\ninternational accounting standards and principles, sufficiency of fund\r\naccounting, and comprehensiveness of data for reporting and compliance\r\nrequirements;\r\n- Ensures contract compliance, including interpreting and monitoring\r\ncontracts with clients, submitting required reports, and monitoring\r\ncovenants and other contract terms;\r\n- Oversees the design, implementation and maintenance of computer-based\r\ninformation system. Oversees records retention (both manual and\r\ncomputer-based) and file maintenance activities;\r\n- Serves as company's risk manager, including evaluating loss exposure\r\nand obtaining insurance as appropriate;\r\n- Manages other administrative operations, such as facilities\r\nmanagement, payroll administration, office operations, and\r\nadministrative support;\r\n- Monitors corporate compliance with by-laws and articles of\r\nincorporation regarding corporate registration and reporting of\r\nfundraising operations.","To perform this job successfully, an\r\nindividual must be able to perform each essential duty satisfactorily.\r\nThe requirements listed below are representative of the knowledge,\r\nskill, and/or ability required.\r\nKnowledge of:\r\n- Generally accepted accounting principles;\r\n- Local accounting standards and legislation;\r\n- State reporting requirements pertaining to accounting;\r\n- Principles and practices of financial management and budgeting;\r\n- Principles and practices of financial systems design and analysis;\r\n- Principles and practices of contract management, records management,\r\nand risk management;\r\n- Principles and practices of management and supervision;\r\n- Principles and practices of information systems management.\r\nAbility to:\r\n- Apply sound fiscal and administrative practices to the company's\r\nactivities;\r\n- Plan, organize and supervise the work of subordinate employees,\r\nincluding training them, assigning and evaluating their work, and\r\nproviding job performance feedback;\r\n- Critically analyze fiscal and administrative policies, practices,\r\nprocedures, and systems, and recommend and implement changes as needed;\r\n- Gather a

In [231]:
skill_df.loc[skill_df['job_id'] == 5760, :]

,job_id,job_description,job_requirement,job_qualification
5752,5760,Unprovided,"- Implement proactive searching and attraction of potential corporate\r\nclients;\r\n- Use individual approach aimed at Clientele retention;\r\n- Study and analyze corporate clients' needs/demands;\r\n- Introduce commercial proposals to corporate clients;\r\n- Organize and conduct presentations on the Company's services,\r\ncontracts signing, sales paperwork processing;\r\n- Realize additional sales of the Company's services to the existing\r\ncorporate clients;\r\n- Submit timely reports in accordance with the internal reporting\r\nsystem;\r\n- Maintain positive and friendly environment in a team and the Company.","- University degree;\r\n- At least 1 year of experience in sales, preferably in FMCG (Fast\r\nMoving Consumer Goods) companies;\r\n- Basic knowledge of sales techniques;\r\n- Knowledge in the field of telecommunication and IT is preferred;\r\n- Reporting and business writing experience;\r\n- Experience in working with external clients;\r\n- Ability and willingness to work in a team;\r\n- Excellent communication and organizational skills;\r\n- Initiative and decision making skills;\r\n- Ability to work under stress;\r\n- Valid driver license is preferred;\r\n- Advanced computer skills: experience in working with MS Office,\r\nPowerPoint;\r\n- Fluency in Armenian, Russian and English languages."


In [232]:
skill_df.loc[skill_df['job_id'] == 9269, :]

,job_id,job_description,job_requirement,job_qualification
9259,9269,Unprovided,"- Develop PHP codes (cross-browser and cross-platform compatible);\r\n- Analyze database using SQL database computer language;\r\n- Analyze existing business processes and make recommendations for the\r\nimprovement and development of new value added processes;\r\n- Produce and maintain clean, high quality standard compliant\r\napplication;\r\n- Perform other duties as assigned by the manager.","- University degree;\r\n- 6 months of professional work experience; \r\n- Knowledge of PHP, HTML/DHTML, XML, CSS and JavaScript, AJAX;\r\n- Knowledge of SQL (MySQL);\r\n- Basic knowledge of Object-oriented programming\r\n- Knowledge of English and Russian languages (basic)."


In [233]:
skill_df.loc[skill_df['job_id'] == 18173, :]

,job_id,job_description,job_requirement,job_qualification
18163,18173,Unprovided,"The\r\njob responsibilities include but are not limited to the following:\r\n- Analyze complex Excel tables and create PowerPoint presentations\r\nwhich include complex charts;\r\n- Promote pages in social media such as Facebook, Twitter, etc.;\r\n- Create media plans and schedules for TV and non-TV placement;\r\n- Conduct media monitoring of the activities of the clients and\r\ntheir competitors","- Proficiency in MS Excel and PowerPoint;\r\n- In depth knowledge of the social media platforms (Facebook,Vkontakte, Odnoklassniki, Twitter etc.);\r\n- Willingness to engage in media, PR, creative development, media\r\nanalysis and client service;\r\n- Strong organizational skills in order to manage multiple tasks\r\nsimultaneously to meet the required deadlines;\r\n- Interest in new advertising trends and techniques;\r\n- Fluency in the Armenian, English and Russian languages;\r\n- Good knowledge of Internet advertising"


## Text preprocessing

In [122]:
# def nltk_tag_to_wordnet_tag(nltk_tag):
#     """Helper function to convert nltk POS tags to wordnet POS tags"""
#     if nltk_tag.startswith("J"):
#         return wordnet.ADJ
#     elif nltk_tag.startswith("V"):
#         return wordnet.VERB
#     elif nltk_tag.startswith("N"):
#         return wordnet.NOUN
#     elif nltk_tag.startswith("R"):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN


# def lemmatize_token(token):
#     lemmatizer = WordNetLemmatizer()
#     nltk_tagged = nltk.pos_tag(token)
#     wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
#     lemmatized_sentence = []
#     for word, tag in wordnet_tagged:
#         if tag is None:
#             # if no tag found then use as it is
#             lemmatized_sentence.append(word)
#         else:
#             # else use the tag to lemmatize the token
#             lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))

#     return " ".join(lemmatized_sentence)


class pre_process_text:
    """Function to pre process text once for all steps:
    - Replace all consecutive special characters by 1 single character "|"
    - Removing URLs
    - Removing stop words
    - Removing special characters and numbers and replace by 1 space
    - Removing short words with length < 2
    - Lowercasing
    - Lemmatizing

    Inputs:
    - text: raw text to normalize
    - spec_chars: list of special characters to remove
    - stop_words: set of stopwords to remove

    Return: normalized text
    """

    def __init__(self, text, spec_chars, stop_words):
        self.text = text
        self.spec_chars = spec_chars
        self.stop_words = stop_words

    def convert2string(self):
        self.text = str(self.text)
        return self

    def lowercase(self):
        # convert to string and lowercasing
        self.text = self.text.lower()
        return self

    def remove_url(self):
        # remove URLs
        self.text = re.sub(r"http\S+", "", self.text)
        self.text = re.sub(r"www\S+", "", self.text)
        return self

    def remove_spec_chars(self):
        # remove special characters and numbers
        # self.text = self.text.translate(str.maketrans("", "", self.spec_chars))
        text = []
        for phrase in self.text:
            for char in self.spec_chars:
                phrase = phrase.replace(char, " ")
            phrase = re.sub("[ ]{2,}", " ", phrase)
            text.append(phrase)
        self.text = text
        return self

    def pad_spec_chars(self):
        # add space before and after special characters if not
        for char in self.spec_chars:
            self.text = self.text.replace(char, "|")
        self.text = re.sub("[|]{1,}", " | ", self.text)
        return self

    def get_tokens(self):
        # tokenize
        self.text = nltk.word_tokenize(self.text)
        return self

    def join_phrases(self):
        # join keyphrases into 1 document
        self.text = [word for phrase in self.text for word in phrase.split()]
        return self

    def remove_stopwords(self):
        # remove stop words
        self.text = [word for word in self.text if word not in self.stop_words]
        #         self.text = [word for word in pre_model(self.text) if word.is_stop is False]
        self.text = " ".join(self.text)
        return self

    def remove_shortwords(self):
        # remove words with length < 2
        self.text = [
            word
            for word in self.text.split()
            if len(word) >= 2 or word in self.spec_chars
        ]
        self.text = " ".join(self.text)
        return self

    def lemmatize(self):
        # lemmatize
        #         self.text = lemmatize_token(self.text)
        # #         lemma_sentence = []
        # #         for token in pre_model(self.text):
        # #             lemma_sentence.append(token.lemma_)
        text = []
        for phrase in self.text:
            phrase = " ".join(
                [word.lemma_ for word in pre_model(phrase) if word.is_stop is False]
            )
            text.append(phrase)
        self.text = text
        return self

    def extract_keyphrases(self):
        # extract keyphrases
        rake_nltk_var = Rake(
            stopwords=self.stop_words,
            punctuations=self.spec_chars,
            min_length=1,
            max_length=5,
        )
        rake_nltk_var.extract_keywords_from_text(self.text)
        self.text = rake_nltk_var.get_ranked_phrases()[:5]
#         text = []
#         for phrase in self.text:
#             words = []
#             for word in phrase.split():
#                 if len(word) >= 2 and word not in self.spec_chars:
#                     words.append(word)
#                 else:
#                     words = words
#             words = " ".join(words)
#         text.append(words)
#         self.text = text
        return self

    def clean_extract_phrase(self):
        self = self.convert2string()
        #         self = self.lowercase()
        self = self.remove_url()
        self = self.pad_spec_chars()
        self = self.remove_shortwords()
        #         self = self.lemmatize()
        self = self.extract_keyphrases()
        self = self.remove_spec_chars()
        return self.text

In [109]:
# load the pre-trained Stanford GloVe model in Word2Vec format (dimension 300)
pre_model = spacy.load('en_core_web_lg')
pre_model.pipe_names

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [110]:
if 'parser' in pre_model.pipe_names:
    pre_model.remove_pipe('parser')
if 'ner' in pre_model.pipe_names:
    pre_model.remove_pipe('ner')
pre_model.pipe_names

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer']

In [123]:
# Set stopwords and special characters to remove
# stop_words = set(stopwords.words("english"))
stop_words = set(STOP_WORDS)
spec_chars = string.punctuation + string.digits + "\n\r"

In [234]:
tqdm.pandas(desc="Job Skill Keyphrase extracting")
clean_skill_df = skill_df[skill_columns].progress_applymap(
    lambda x: pre_process_text(
        x, spec_chars=spec_chars, stop_words=stop_words
    ).clean_extract_phrase()
)
clean_skill_df.insert(0, "job_id", skill_df["job_id"])
clean_skill_df.head(3)

Job Skill Keyphrase extracting: 100%|██████████████████████████████████████████| 56976/56976 [00:18<00:00, 3072.94it/s]


,job_id,job_description,job_requirement,job_qualification
0,1,"[ameria investment consulting company, requires high level, provides highly responsible, complex staff assistance, chief financial officer]","[liquidity reserves assists investment team, including evaluating loss exposure, internal financial management reports, investment team leader, management team participating]","[providing job performance feedback, operate specialized software applications, state reporting requirements pertaining, generally accepted accounting principles, accounting technical skills coupled]"
1,2,"[irex currently seeks, position reports directly, cc program manager, yerevan office, community connections]","[completing general administrative tasks, includes coordinating logistics, cc filing system, cc staff, cc program]","[public speaking skills, excellent skills, work independently, good communication, written english]"
2,3,"[time position, primary contact, international organizations, environmental ngos, public outreach]","[regular electronic communications, provide environmental information, primary local contact, prepare cenn seminars, international organizations]","[environmentally related field, environmental issues specific, years relevant, written fluency, salary commensurate]"


In [235]:
clean_skill_df.loc[clean_skill_df['job_id'] == 104, :]

,job_id,job_description,job_requirement,job_qualification
103,104,"[seeking qualified candidate, position, polymer, organic, fill]","[scientific team, prepare membranes, work, members, develop]","[english language, advanced degree, russian, remuneration, knowledge]"


In [236]:
clean_skill_df.loc[clean_skill_df['job_id'] == 216, :]

,job_id,job_description,job_requirement,job_qualification
213,216,"[regional security office, yerevan, seeking, position, individual]","[complete position description listing, human resources office, authorized personnel enter, performs guard duties, usg employees]","[current employees serving probationary period, valid drivers license, technical security equipment, relevant work experience, hold fma appointment]"


In [237]:
clean_skill_df.loc[clean_skill_df['job_id'] == 5694, :]

,job_id,job_description,job_requirement,job_qualification
5686,5694,[unprovided],"[produce branding elements, create graphic elements, work closely, web development, sales teams]","[written strong communication skills, started conceptual thinker, relevant higher education, meet strict deadlines, relevant work experience]"


In [238]:
clean_skill_df.loc[clean_skill_df['job_id'] == 5760, :]

,job_id,job_description,job_requirement,job_qualification
5752,5760,[unprovided],"[use individual approach aimed, submit timely reports, sales paperwork processing, realize additional sales, introduce commercial proposals]","[valid driver license, moving consumer goods, decision making skills, advanced computer skills, business writing experience]"


In [239]:
clean_skill_df.loc[clean_skill_df['job_id'] == 18173, :]

,job_id,job_description,job_requirement,job_qualification
18163,18173,[unprovided],"[analyze complex excel tables, include complex charts, job responsibilities include, create powerpoint presentations, create media plans]","[strong organizational skills, manage multiple tasks, new advertising trends, social media platforms, internet advertising]"


In [240]:
tqdm.pandas(desc="Job Skill Lemmatizing")
lemma_skill_df = clean_skill_df[skill_columns].progress_applymap(
    lambda x: pre_process_text(x, spec_chars=spec_chars, stop_words=stop_words)
    .lemmatize().text
)
lemma_skill_df.insert(0, "job_id", skill_df["job_id"])
lemma_skill_df.head(3)

Job Skill Lemmatizing: 100%|████████████████████████████████████████████████████| 56976/56976 [09:22<00:00, 101.26it/s]


,job_id,job_description,job_requirement,job_qualification
0,1,"[ameria investment consult company, require high level, provide highly responsible, complex staff assistance, chief financial officer]","[liquidity reserve assist investment team, include evaluate loss exposure, internal financial management report, investment team leader, management team participate]","[provide job performance feedback, operate specialized software application, state reporting requirement pertain, generally accept accounting principle, account technical skill couple]"
1,2,"[irex currently seek, position report directly, cc program manager, yerevan office, community connection]","[complete general administrative task, include coordinate logistic, cc file system, cc staff, cc program]","[public speaking skill, excellent skill, work independently, good communication, write english]"
2,3,"[time position, primary contact, international organization, environmental ngo, public outreach]","[regular electronic communication, provide environmental information, primary local contact, prepare cenn seminar, international organization]","[environmentally relate field, environmental issue specific, year relevant, write fluency, salary commensurate]"


In [241]:
lemma_skill_df.loc[lemma_skill_df['job_id'] == 104, :]

,job_id,job_description,job_requirement,job_qualification
103,104,"[seek qualified candidate, position, polymer, organic, fill]","[scientific team, prepare membrane, work, member, develop]","[english language, advanced degree, russian, remuneration, knowledge]"


In [242]:
lemma_skill_df.loc[lemma_skill_df['job_id'] == 216, :]

,job_id,job_description,job_requirement,job_qualification
213,216,"[regional security office, yerevan, seek, position, individual]","[complete position description list, human resource office, authorize personnel enter, perform guard duty, usg employee]","[current employee serve probationary period, valid driver license, technical security equipment, relevant work experience, hold fma appointment]"


In [243]:
lemma_skill_df.loc[lemma_skill_df['job_id'] == 5694, :]

,job_id,job_description,job_requirement,job_qualification
5686,5694,[unprovide],"[produce branding element, create graphic element, work closely, web development, sale team]","[write strong communication skill, start conceptual thinker, relevant high education, meet strict deadline, relevant work experience]"


In [244]:
lemma_skill_df.loc[lemma_skill_df['job_id'] == 5760, :]

,job_id,job_description,job_requirement,job_qualification
5752,5760,[unprovide],"[use individual approach aim, submit timely report, sale paperwork processing, realize additional sale, introduce commercial proposal]","[valid driver license, move consumer good, decision make skill, advanced computer skill, business writing experience]"


In [245]:
lemma_skill_df.loc[lemma_skill_df['job_id'] == 18173, :]

,job_id,job_description,job_requirement,job_qualification
18163,18173,[unprovide],"[analyze complex excel table, include complex chart, job responsibility include, create powerpoint presentation, create medium plan]","[strong organizational skill, manage multiple task, new advertising trend, social medium platform, internet advertising]"


In [246]:
lemma_skill_df.rename(
    columns={
        "job_description": "job_description_lemma",
        "job_requirement": "job_requirement_lemma",
        "job_qualification": "job_qualification_lemma",
    },
    inplace=True,
)
lemma_skill_df.head(3)

,job_id,job_description_lemma,job_requirement_lemma,job_qualification_lemma
0,1,"[ameria investment consult company, require high level, provide highly responsible, complex staff assistance, chief financial officer]","[liquidity reserve assist investment team, include evaluate loss exposure, internal financial management report, investment team leader, management team participate]","[provide job performance feedback, operate specialized software application, state reporting requirement pertain, generally accept accounting principle, account technical skill couple]"
1,2,"[irex currently seek, position report directly, cc program manager, yerevan office, community connection]","[complete general administrative task, include coordinate logistic, cc file system, cc staff, cc program]","[public speaking skill, excellent skill, work independently, good communication, write english]"
2,3,"[time position, primary contact, international organization, environmental ngo, public outreach]","[regular electronic communication, provide environmental information, primary local contact, prepare cenn seminar, international organization]","[environmentally relate field, environmental issue specific, year relevant, write fluency, salary commensurate]"


## Transform to skill-driven data

In [247]:
tmp1 = clean_skill_df[["job_id", "job_description"]].explode("job_description")

tmp1["skill_id"] = tmp1.groupby("job_id")["job_id"].rank(method="first", ascending=True)
tmp1["column_type"] = "job_description"
tmp1.rename(columns={"job_description": "skill"}, inplace=True)
tmp1.iloc[:20, ]

,job_id,skill,skill_id,column_type
0,1,ameria investment consulting company,1.0,job_description
0,1,requires high level,2.0,job_description
0,1,provides highly responsible,3.0,job_description
0,1,complex staff assistance,4.0,job_description
0,1,chief financial officer,5.0,job_description
1,2,irex currently seeks,1.0,job_description
1,2,position reports directly,2.0,job_description
1,2,cc program manager,3.0,job_description
1,2,yerevan office,4.0,job_description
1,2,community connections,5.0,job_description


In [248]:
tmp2 = clean_skill_df[["job_id", "job_requirement"]].explode("job_requirement")

tmp2["skill_id"] = tmp2.groupby("job_id")["job_id"].rank(
    method="first", ascending=True
)
tmp2["column_type"] = "job_requirement"
tmp2.rename(columns={"job_requirement": "skill"}, inplace=True)
tmp2.iloc[:20, ]

,job_id,skill,skill_id,column_type
0,1,liquidity reserves assists investment team,1.0,job_requirement
0,1,including evaluating loss exposure,2.0,job_requirement
0,1,internal financial management reports,3.0,job_requirement
0,1,investment team leader,4.0,job_requirement
0,1,management team participating,5.0,job_requirement
1,2,completing general administrative tasks,1.0,job_requirement
1,2,includes coordinating logistics,2.0,job_requirement
1,2,cc filing system,3.0,job_requirement
1,2,cc staff,4.0,job_requirement
1,2,cc program,5.0,job_requirement


In [249]:
tmp3 = clean_skill_df[["job_id", "job_qualification"]].explode("job_qualification")

tmp3["skill_id"] = tmp3.groupby("job_id")["job_id"].rank(method="first", ascending=True)
tmp3["column_type"] = "job_qualification"
tmp3.rename(columns={"job_qualification": "skill"}, inplace=True)
tmp3.iloc[:20, ]

,job_id,skill,skill_id,column_type
0,1,providing job performance feedback,1.0,job_qualification
0,1,operate specialized software applications,2.0,job_qualification
0,1,state reporting requirements pertaining,3.0,job_qualification
0,1,generally accepted accounting principles,4.0,job_qualification
0,1,accounting technical skills coupled,5.0,job_qualification
1,2,public speaking skills,1.0,job_qualification
1,2,excellent skills,2.0,job_qualification
1,2,work independently,3.0,job_qualification
1,2,good communication,4.0,job_qualification
1,2,written english,5.0,job_qualification


In [250]:
skill_long_df = tmp1.append(tmp2)
skill_long_df = skill_long_df.append(tmp3)
(
    skill_long_df.shape,
    skill_long_df.iloc[
        :10,
    ],
    skill_long_df.iloc[
        100000:100010,
    ],
    skill_long_df.iloc[
        -10:,
    ],
)

((257205, 4),
    job_id                                 skill  skill_id      column_type
 0       1  ameria investment consulting company       1.0  job_description
 0       1                   requires high level       2.0  job_description
 0       1           provides highly responsible       3.0  job_description
 0       1              complex staff assistance       4.0  job_description
 0       1               chief financial officer       5.0  job_description
 1       2                  irex currently seeks       1.0  job_description
 1       2             position reports directly       2.0  job_description
 1       2                    cc program manager       3.0  job_description
 1       2                        yerevan office       4.0  job_description
 1       2                 community connections       5.0  job_description,
       job_id                                       skill  skill_id  \
 4923    4931                 major situation threatening       3.0   
 4923  

In [251]:
column_ids, column_types = pd.factorize(skill_long_df['column_type'])

In [252]:
column_ids, column_types

(array([0, 0, 0, ..., 2, 2, 2], dtype=int64),
 Index(['job_description', 'job_requirement', 'job_qualification'], dtype='object'))

In [253]:
skill_long_df["column_id"] = column_ids + 1
(
    skill_long_df.shape,
    skill_long_df.iloc[
        :10,
    ],
    skill_long_df.iloc[
        100000:100010,
    ],
    skill_long_df.iloc[
        -10:,
    ],
)

((257205, 5),
    job_id                                 skill  skill_id      column_type  \
 0       1  ameria investment consulting company       1.0  job_description   
 0       1                   requires high level       2.0  job_description   
 0       1           provides highly responsible       3.0  job_description   
 0       1              complex staff assistance       4.0  job_description   
 0       1               chief financial officer       5.0  job_description   
 1       2                  irex currently seeks       1.0  job_description   
 1       2             position reports directly       2.0  job_description   
 1       2                    cc program manager       3.0  job_description   
 1       2                        yerevan office       4.0  job_description   
 1       2                 community connections       5.0  job_description   
 
    column_id  
 0          1  
 0          1  
 0          1  
 0          1  
 0          1  
 1          1  
 1 

In [254]:
tmp1 = lemma_skill_df[["job_id", "job_description_lemma"]].explode(
    "job_description_lemma"
)
tmp1["skill_id"] = tmp1.groupby("job_id")["job_id"].rank(method="first", ascending=True)
tmp1["column_type"] = "job_description"
tmp1.rename(columns={"job_description_lemma": "skill_lemma"}, inplace=True)

tmp2 = lemma_skill_df[["job_id", "job_requirement_lemma"]].explode(
    "job_requirement_lemma"
)
tmp2["skill_id"] = tmp2.groupby("job_id")["job_id"].rank(method="first", ascending=True)
tmp2["column_type"] = "job_requirement"
tmp2.rename(columns={"job_requirement_lemma": "skill_lemma"}, inplace=True)

tmp3 = lemma_skill_df[["job_id", "job_qualification_lemma"]].explode(
    "job_qualification_lemma"
)
tmp3["skill_id"] = tmp3.groupby("job_id")["job_id"].rank(method="first", ascending=True)
tmp3["column_type"] = "job_qualification"
tmp3.rename(columns={"job_qualification_lemma": "skill_lemma"}, inplace=True)

In [255]:
lemma_skill_long_df = tmp1.append(tmp2)
lemma_skill_long_df = lemma_skill_long_df.append(tmp3)
(
    lemma_skill_long_df.shape,
    lemma_skill_long_df.iloc[
        :10,
    ],
    lemma_skill_long_df.iloc[
        100000:100010,
    ],
    lemma_skill_long_df.iloc[
        -10:,
    ],
)

((257205, 4),
    job_id                        skill_lemma  skill_id      column_type
 0       1  ameria investment consult company       1.0  job_description
 0       1                 require high level       2.0  job_description
 0       1         provide highly responsible       3.0  job_description
 0       1           complex staff assistance       4.0  job_description
 0       1            chief financial officer       5.0  job_description
 1       2                irex currently seek       1.0  job_description
 1       2           position report directly       2.0  job_description
 1       2                 cc program manager       3.0  job_description
 1       2                     yerevan office       4.0  job_description
 1       2               community connection       5.0  job_description,
       job_id                                 skill_lemma  skill_id  \
 4923    4931                    major situation threaten       3.0   
 4923    4931                    perform

In [256]:
# lemma_skill_long_df.reset_index(drop=True)
# skill_long_df.reset_index(drop=True)

clean_skill_long_df = pd.merge(
    lemma_skill_long_df,
    skill_long_df,
    how="inner",
    on=["job_id", "skill_id", "column_type"],
)
clean_skill_long_df = clean_skill_long_df[
    ["job_id", "skill_id", "column_id", "column_type", "skill", "skill_lemma"]
]
print(clean_skill_long_df.shape)
clean_skill_long_df.iloc[
    :10,
]

(257205, 6)


,job_id,skill_id,column_id,column_type,skill,skill_lemma
0,1,1.0,1,job_description,ameria investment consulting company,ameria investment consult company
1,1,2.0,1,job_description,requires high level,require high level
2,1,3.0,1,job_description,provides highly responsible,provide highly responsible
3,1,4.0,1,job_description,complex staff assistance,complex staff assistance
4,1,5.0,1,job_description,chief financial officer,chief financial officer
5,2,1.0,1,job_description,irex currently seeks,irex currently seek
6,2,2.0,1,job_description,position reports directly,position report directly
7,2,3.0,1,job_description,cc program manager,cc program manager
8,2,4.0,1,job_description,yerevan office,yerevan office
9,2,5.0,1,job_description,community connections,community connection


In [257]:
tqdm.pandas(desc="Job Skill Converting ID to integer")
clean_skill_long_df["skill_id"] = clean_skill_long_df["skill_id"].progress_apply(
    lambda x: int(x)
)
clean_skill_long_df.iloc[:25, ]

Job Skill Converting ID to integer: 100%|██████████████████████████████████| 257205/257205 [00:00<00:00, 685939.75it/s]


,job_id,skill_id,column_id,column_type,skill,skill_lemma
0,1,1,1,job_description,ameria investment consulting company,ameria investment consult company
1,1,2,1,job_description,requires high level,require high level
2,1,3,1,job_description,provides highly responsible,provide highly responsible
3,1,4,1,job_description,complex staff assistance,complex staff assistance
4,1,5,1,job_description,chief financial officer,chief financial officer
5,2,1,1,job_description,irex currently seeks,irex currently seek
6,2,2,1,job_description,position reports directly,position report directly
7,2,3,1,job_description,cc program manager,cc program manager
8,2,4,1,job_description,yerevan office,yerevan office
9,2,5,1,job_description,community connections,community connection


In [258]:
clean_skill_long_df.shape

(257205, 6)

In [259]:
clean_skill_long_df.loc[76310:76320, :]

,job_id,skill_id,column_id,column_type,skill,skill_lemma
76310,18172,5,1,job_description,business english courses,business english course
76311,18173,1,1,job_description,unprovided,unprovide
76312,18174,1,1,job_description,country office refugee assistance,country office refugee assistance
76313,18174,2,1,job_description,underserved communities populated,underserved community populate
76314,18174,3,1,job_description,sc refugee projects,sc refugee project
76315,18174,4,1,job_description,refugee programs,refugee program
76316,18174,5,1,job_description,project coordinator,project coordinator
76317,18175,1,1,job_description,unprovided,unprovide
76318,18176,1,1,job_description,enhancing economic ties,enhance economic tie
76319,18176,2,1,job_description,usaid funded project,usaid funded project


In [260]:
tqdm.pandas(desc="Job Skill word tokenizing")
clean_skill_long_df["skill_token"] = clean_skill_long_df["skill_lemma"].progress_apply(
    lambda x: nltk.word_tokenize(x)
)
clean_skill_long_df

Job Skill word tokenizing: 100%|████████████████████████████████████████████| 257205/257205 [00:20<00:00, 12601.66it/s]


,job_id,skill_id,column_id,column_type,skill,skill_lemma,skill_token
0,1,1,1,job_description,ameria investment consulting company,ameria investment consult company,"[ameria, investment, consult, company]"
1,1,2,1,job_description,requires high level,require high level,"[require, high, level]"
2,1,3,1,job_description,provides highly responsible,provide highly responsible,"[provide, highly, responsible]"
3,1,4,1,job_description,complex staff assistance,complex staff assistance,"[complex, staff, assistance]"
4,1,5,1,job_description,chief financial officer,chief financial officer,"[chief, financial, officer]"
...,...,...,...,...,...,...,...
257200,19001,1,3,job_qualification,ra financial system,ra financial system,"[ra, financial, system]"
257201,19001,2,3,job_qualification,higher legal education,high legal education,"[high, legal, education]"
257202,19001,3,3,job_qualification,high pressure environment,high pressure environment,"[high, pressure, environment]"
257203,19001,4,3,job_qualification,professional work experience,professional work experience,"[professional, work, experience]"


In [261]:
clean_skill_long_df["row_id"] = (
    clean_skill_long_df["job_id"].astype(str).str.zfill(5)
    + clean_skill_long_df["skill_id"].astype(str).str.zfill(2)
    + clean_skill_long_df["column_id"].astype(str).str.zfill(2)
)
clean_skill_long_df["key_id"] = (
    clean_skill_long_df["job_id"].astype(str).str.zfill(5)
    + clean_skill_long_df["skill_id"].astype(str).str.zfill(2)
)
clean_skill_long_df = clean_skill_long_df[
    [
        "row_id",
        "key_id",
        "job_id",
        "skill_id",
        "column_id",
        "column_type",
        "skill",
        "skill_lemma",
        "skill_token",
    ]
]
clean_skill_long_df

,row_id,key_id,job_id,skill_id,column_id,column_type,skill,skill_lemma,skill_token
0,000010101,0000101,1,1,1,job_description,ameria investment consulting company,ameria investment consult company,"[ameria, investment, consult, company]"
1,000010201,0000102,1,2,1,job_description,requires high level,require high level,"[require, high, level]"
2,000010301,0000103,1,3,1,job_description,provides highly responsible,provide highly responsible,"[provide, highly, responsible]"
3,000010401,0000104,1,4,1,job_description,complex staff assistance,complex staff assistance,"[complex, staff, assistance]"
4,000010501,0000105,1,5,1,job_description,chief financial officer,chief financial officer,"[chief, financial, officer]"
...,...,...,...,...,...,...,...,...,...
257200,190010103,1900101,19001,1,3,job_qualification,ra financial system,ra financial system,"[ra, financial, system]"
257201,190010203,1900102,19001,2,3,job_qualification,higher legal education,high legal education,"[high, legal, education]"
257202,190010303,1900103,19001,3,3,job_qualification,high pressure environment,high pressure environment,"[high, pressure, environment]"
257203,190010403,1900104,19001,4,3,job_qualification,professional work experience,professional work experience,"[professional, work, experience]"


## Save pre-processed text to csv file

In [262]:
# Save cleaned job skills in file
clean_skill_df_outfile = OUTPUT_DIR + "clean_skill_df.csv"
clean_skill_df.to_csv(clean_skill_df_outfile, index=False)

In [263]:
# Save lemmatized job skills in file
lemma_skill_df_outfile = OUTPUT_DIR + "lemma_skill_df.csv"
lemma_skill_df.to_csv(lemma_skill_df_outfile, index=False)

In [264]:
# Save cleaned job skill tokens in file
skill_tokens_long_df_outfile = OUTPUT_DIR + "skill_tokens_long_df.csv"
clean_skill_long_df.to_csv(skill_tokens_long_df_outfile, index=False)